# Limpieza de datos

**Principal inconveniente:** Los codigos del proyecto anterior actualmente no se pueden utilizar para esta encuesta. Se deben realizar un codigo adaptado el formato de la nueva encuesta.

## Tareas de limpieza:

### Para cada tabla por separado

1. Revisar tipo de pregunta (opcion unica o multiple) 
1. Revisar si la pregunta tiene opcion de "otro" / detalle
1. Si es opcion multiple considerar las columnas adicionales 
1. Revisar si la pregunta tiene un criterio de relevancia
1. Si tiene criterio de relevancia colocar "No valido" donde corresponda (Reducir valores perdidos).
1. Revisar categorias una por una y limpiar las respuestas posibles por ejemplo: "papa", "Papa", --> "Papa" 

### Especificamente para tablas secundarias 

- Agrupar las preguntas de las tablas secundarias en columnas para la tabla principial (segun lo coordinado con Renata).
- Agregar "filas" secundaria como nuevas columnas (pregunta_A_1, pregunta_A_2, ...) para cada observacion de la tabla principal 

### Extra

- Ahora si con los datos limpios se pueden realizar los reportes preliminares para compartir con el equipo de investigacion. 

### Tiempo estimado: 

- 14 (1/13) tablas a 2 o 3 tablas/dia = 7 o 5 dias

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.notebook import tqdm
from pandas_profiling import ProfileReport

In [4]:
%matplotlib inline

In [5]:
tqdm.pandas()

## Read Data

In [6]:
DATA_FP = "inputs/encuesta_principal/Vecindarios_Alimentarios_Encuesta_Integrada_2022_11_23.xlsx"

In [7]:
df_main = pd.read_excel(DATA_FP, sheet_name=0) # Main sheet
df_comp_hogar = pd.read_excel(DATA_FP, sheet_name="composicion_hogar")
df_med_control_plagas = pd.read_excel(DATA_FP, sheet_name="_5_18_medidas_control_plagas")
df_prod_control_plagas = pd.read_excel(DATA_FP, sheet_name="_5_19_productos_control_plagas")
df_control_maleza = pd.read_excel(DATA_FP, sheet_name="_5_20_control_malezas")
df_oferta_alim_prod = pd.read_excel(DATA_FP, sheet_name="_7_1_Oferta_Alimentos_Produccio")
df_productor_dejo_prod_vend = pd.read_excel(DATA_FP, sheet_name="_7_2_produc_dejo_producir_vend")
df_productos_vende_canal = pd.read_excel(DATA_FP, sheet_name="Productos_Vende_Canal")
df_productos_compra_canal = pd.read_excel(DATA_FP, sheet_name="Productos_Compra_Canal")
df_canales_venta_productores = pd.read_excel(DATA_FP, sheet_name="canales_venta_productores")
df_aplic_bioprep = pd.read_excel(DATA_FP, sheet_name="aplicacion_biopreparados")
df_per_trab_puesto_venta = pd.read_excel(DATA_FP, sheet_name="personas_trabajan_puesto_venta")
df_canales_compra_comerciantes = pd.read_excel(DATA_FP, sheet_name="canales_compra_comerciantes")
df_huerto_rel = pd.read_excel(DATA_FP, sheet_name="HUERTO_RELACIONES")

In [8]:
DATA_APOYO_FP = "inputs/encuesta_principal/Datos_Apoyos_Cat_Vec_Alim_2022_12_04.xlsx"

In [9]:
df_apoyo_recibido = pd.read_excel(DATA_APOYO_FP, sheet_name="APOYO RECIBIDO")
df_apoyo_brindado = pd.read_excel(DATA_APOYO_FP, sheet_name="APOYO BRINDADO")

In [14]:
dfs = {
    "Principal": df_main,
    "composicion_hogar": df_comp_hogar,
    "_5_18_medidas_control_plagas": df_med_control_plagas,
    "_5_19_productos_control_plagas": df_prod_control_plagas,
    "_5_20_control_malezas": df_control_maleza,
    "_7_1_Oferta_Alimentos_Produccio": df_oferta_alim_prod,
    "_7_2_produc_dejo_producir_vend": df_productor_dejo_prod_vend,
    "Productos_Vende_Canal": df_productos_vende_canal,
    "canales_venta_productores": df_canales_venta_productores,
    "aplicacion_biopreparados": df_aplic_bioprep,
    "personas_trabajan_puesto_venta": df_per_trab_puesto_venta,
    "canales_compra_comerciantes": df_canales_compra_comerciantes,
    "Productos_Compra_Canal": df_productos_compra_canal,
    "APOYO RECIBIDO": df_apoyo_recibido,
    "APOYO BRINDADO": df_apoyo_brindado,
    "HUERTO_RELACIONES": df_huerto_rel
}

## Generacion de reportes de la tabla principal

Los reportes seran divididos por Ciudad / Vecindario / Perfil y Perfil

In [12]:
for col in ["_1_4_1_ciudad", "_1_4_2_vecindario", "_1_5_perfil_entrevistado"]:
    print(f"Valores unicos de {col}")
    print(df_main[col].value_counts())
    print("-"*30)

Valores unicos de _1_4_1_ciudad
lima     492
quito    400
Name: _1_4_1_ciudad, dtype: int64
------------------------------
Valores unicos de _1_4_2_vecindario
carabayllo      273
solanda         217
pachacamac      213
la_magdalena    159
fuera            32
Name: _1_4_2_vecindario, dtype: int64
------------------------------
Valores unicos de _1_5_perfil_entrevistado
consumidor     656
productor      149
comerciante     90
Name: _1_5_perfil_entrevistado, dtype: int64
------------------------------


In [13]:
for ciudad in tqdm(df_main["_1_4_1_ciudad"].dropna().unique()):
    dff_ciudad = df_main[df_main["_1_4_1_ciudad"] == ciudad]
    profile = ProfileReport(dff_ciudad, title=f"Principal-{ciudad}", minimal=True)
    
    output_fp = f"outputs/data_cleaning/reports/Principal-{ciudad}.html"
    if not os.path.exists(output_fp): profile.to_file(output_fp)

    for vecindario in tqdm(dff_ciudad["_1_4_2_vecindario"].unique()):
        if vecindario != "fuera":
            dff_vecindario = dff_ciudad[dff_ciudad["_1_4_2_vecindario"] == vecindario]
            profile = ProfileReport(dff_vecindario, title=f"Principal-{vecindario}", minimal=True)
            
            output_fp = f"outputs/data_cleaning/reports/Principal-{vecindario}.html"
            if not os.path.exists(output_fp): profile.to_file(output_fp)
            
            for perfil in tqdm(dff_vecindario["_1_5_perfil_entrevistado"].unique()):
                dff_perfil = dff_vecindario[dff_vecindario["_1_5_perfil_entrevistado"] == perfil]
                profile = ProfileReport(dff_perfil, title=f"Principal-{vecindario}-{perfil}", minimal=True)
        
                output_fp = f"outputs/data_cleaning/reports/Principal-{vecindario}-{perfil}.html"
                if not os.path.exists(output_fp): profile.to_file(output_fp)
    
    for perfil in tqdm(dff_ciudad["_1_5_perfil_entrevistado"].unique()):
        dff_perfil = dff_ciudad[dff_ciudad["_1_5_perfil_entrevistado"] == perfil]
        profile = ProfileReport(dff_perfil, title=f"Principal-{ciudad}-{perfil}", minimal=True)
        
        output_fp = f"outputs/data_cleaning/reports/Principal-{ciudad}-{perfil}.html"
        if not os.path.exists(output_fp): profile.to_file(output_fp)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
for name, df in tqdm(dfs.items()):
    output_fp = f"outputs/data_cleaning/reports/{name}.html"
    if not os.path.exists(output_fp):
        try:
            profile = ProfileReport(df, 
                                    title=name, 
                                    minimal=True)
            profile.to_file(output_fp)
        except:
            pass

  0%|          | 0/16 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Datos Generales del Cuestionario

In [ ]:
col_name = "_1_5_perfil_entrevistado"

def frec_table(df, col_name):
    col_value_counts = pd.concat([
        df_main[col_name].value_counts(),
        df_main[col_name].value_counts(normalize=True).round(2) * 100,
    ], axis=1)
    col_value_counts.index = col_value_counts.index.str.capitalize()
    col_value_counts.columns = ["Frec.", "Frec. Rel. (%)"]

    col_value_counts = pd.concat([
        col_value_counts, 
        col_value_counts.sum().to_frame(name="Total").T
    ])

    return col_value_counts

In [ ]:
pd.crosstab(df_main["_1_5_perfil_entrevistado"],
            df_main["_1_4_1_ciudad"],
            margins=True, margins_name="Total")

In [ ]:
# Frecs. de encuestas por vecindario y ciudad 
vec_x_ciudad = (df_main
    .groupby(["_1_4_1_ciudad", "_1_4_2_vecindario"])
    .size()
    .to_frame()
)

totals = df_main["_1_4_1_ciudad"].value_counts()
normalized = pd.concat([
    vec_x_ciudad.loc["lima"] / totals["lima"],
    vec_x_ciudad.loc["quito"] / totals["quito"]
]).round(2) * 100
vec_x_ciudad[1] = normalized[0].values

vec_x_ciudad.columns = ["Frec.", 'Frec. Rel. (%)']
vec_x_ciudad

In [ ]:
perfil_x_vec = (df_main
    .groupby(["_1_4_1_ciudad", "_1_4_2_vecindario", "_1_5_perfil_entrevistado"])
    .size()
    .to_frame()
)
perfil_x_vec[1] = (perfil_x_vec[0] / df_main.groupby(["_1_4_1_ciudad", "_1_4_2_vecindario"]).size()).round(2) * 100
perfil_x_vec.columns = ["Frec.", 'Frec. Rel. (%)']
perfil_x_vec

In [ ]:
# Num de encuestas diarias
(df_main
    .groupby("_1_2_fecha_encuesta").size()
    .resample("D").sum()
    .plot(style="-", figsize=(10, 5))
)

In [ ]:
fig, axs = plt.subplots(figsize=(12, 4))
(df_main
    .groupby(df_main["_1_2_fecha_encuesta"]
    .dt.day_name('es_ES.utf8'))
    .size()
    .plot(kind='bar', rot=0, ax=axs))
plt.xlabel("Dia de la semana");
plt.ylabel("Encuestas");

In [ ]:
df_main.columns[5:]

In [ ]:
df_main[['zona_carabayllo',
       'otra_zona_carabayllo', 'zona_pachacamac', 'otra_zona_pachacamac',
       'mercado_carabayllo',]].isna().sum()

In [ ]:
df_main.columns[:40]

In [ ]:
df_main[['zona_carabayllo', 'otra_zona_carabayllo']].dropna(how='all').isna().sum()

In [ ]:
df_main['_1_4_2_vecindario'].value_counts()['carabayllo']

In [ ]:
df_main['zona_carabayllo'].value_counts()

In [ ]:
df_main['otra_zona_carabayllo'].value_counts()

In [ ]:
df_main['otra_zona_carabayllo'].value_counts().sum()

In [ ]:
df_main[['mercado_carabayllo', 'otro_mercado_carabayllo']].dropna(how='all')

In [ ]:
'_1_1_codigo_cuestionario',

'_1_2_fecha_encuesta', 

'_1_3_nombre_encuestador', 


'_1_4_1_ciudad', '_1_4_2_vecindario', 'sector_barrio_huerto', 
'_1_5_perfil_entrevistado',
       
    
    
    
'zona_carabayllo', 'otra_zona_carabayllo', 
'mercado_carabayllo', 'otro_mercado_carabayllo',

'zona_pachacamac', 'otra_zona_pachacamac', 
'mercado_pachacamac', 'otro_mercado_pachacamac',
       
    
'_1_6_ambito_territorial', '_1_7_1_provincia_huerto'
'_1_7_2_canton_huerto', '_1_7_3_parroquia_huerto',
       
    
'_2_1_consentimiento_informado', 
'_3_1_Nombre', 'sexo', 'edad',
'aporta_economicamente_hogar', '_3_2_Informo_telefono', 
'numero_telefono', 'contacto_dirigente', 'contacto_administrador',
'_3_3_1_Distrito_Canton', '_3_3_2_Provincia', '_3_3_3_Pais',
'otro_pais', '_3_4_tiempo_vive_hogar', '_3_4_Vive_vecindario',
'_3_6_nivel_estudio', 'nivel_completo', 'anos_nivel_incompleto'

In [ ]:
sexo	edad	aporta_economicamente_hogar
num_hombres_hogar	num_menores_hogar	num_miembros_aportan_hogar


In [ ]:
df_main

## Caracterizacion del encuestado

In [17]:
df_main["_3_2_Informo_telefono"].value_counts()

si    522
no    352
Name: _3_2_Informo_telefono, dtype: int64

### 3.3 Lugar de nacimiento

In [21]:
df_main["_3_3_3_Pais"].value_counts()

peru       481
ecuador    391
otro        15
Name: _3_3_3_Pais, dtype: int64

In [22]:
df_main["_3_3_2_Provincia"].value_counts()

Pichincha                   216
Lima                        176
LIMA                         81
Quito                        27
Cotopaxi                     18
                           ... 
Orellana                      1
Guayaqui                      1
Huanta                        1
Tocache                       1
Santo Domingo de Olleros      1
Name: _3_3_2_Provincia, Length: 178, dtype: int64

In [32]:
df_main["_1_4_1_ciudad"].value_counts().sum()

892

In [34]:
df_main[df_main["_1_4_1_ciudad"].isna()]

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_altitude,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_precision,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags
186,187,2022-08-22 10:02:09.332,2022-08-22 10:19:02.548,NaN,2022-08-16,Julian Ordoñez,NaN,solanda,NaN,consumidor,...,NaN,NaN,177686949,c3e2305c-e339-44bb-8c67-470f8c72684c,2022-08-22 15:18:04,NaN,NaN,submitted_via_web,funsad,NaN
414,415,2022-08-30 23:42:59.163,2022-08-30 23:43:25.236,NaN,NaT,NaN,NaN,NaN,NaN,consumidor,...,NaN,NaN,180041604,94e0b9d7-3cbe-4fa9-8226-25f9e007a9b3,2022-08-31 04:43:37,NaN,NaN,submitted_via_web,funsad,NaN
884,885,2022-10-19 17:48:15.178,2022-10-29 14:34:46.026,NaN,2022-10-03,Anny Gomez de la Cruz,NaN,pachacamac,NaN,productor,...,NaN,NaN,193631251,0618a2ab-7301-4251-b2e3-0b8c8ffbbce0,2022-10-19 23:33:08,NaN,NaN,submitted_via_web,jorgereyna,NaN


In [37]:
df_main.loc[186, ["_1_4_1_ciudad", "_1_4_2_vecindario"]]

_1_4_1_ciudad            NaN
_1_4_2_vecindario    solanda
Name: 186, dtype: object

In [38]:
df_main.loc[186, "_1_4_1_ciudad"] = "quito"

In [80]:
df_main.loc[186, "_1_4_1_ciudad"]

'quito'

In [82]:
df_main["_1_4_1_ciudad"].isna().sum()

1

In [49]:
df_main.shape

(895, 825)

# Variables Productores
    a_2019_area_produccion
    _1_6_ambito_territorial
    _4_1_principal_actividad_econo
    ingreso_hogar
    _5_Caracteriza_Actividad_Econo
    especificas_productores
    INVERSION_INGRESOS_PRODUCCION
    Porcentaje_Autoconsumo
    _6_6_Dificultades_Comerciales
    _7_RELACIONES_VENTA_ALIMENTOS
    _7_1_Oferta_Alimentos_Produccio
    _7_2_produc_dejo_producir_vend
    _7_5_canal_venta_productores
    _9_2_relaciones_huertos
    aplica_biopreparados
    _10_3_sufrio_lesiones_golpes
    _10_4_sintomas_intoxicacion
    _10_7_productor_problemas_paro
    problemas_paro
    resolucion_problema_paro


# Variables Comerciantes
    mercado_carabayllo
    mercado_pachacamac
    contacto_administrador
    _3_4_Vive_vecindario
    _4_1_principal_actividad_econo
    informacion_puesto_comerciante
    _5_Caracteriza_Actividad_Econo
    especificas_comercianes
    INVERSION_INGRESOS_PRODUCCION
    CAMBIO_INGRESOS
    _6_4_FINANCIA_COSTOS_PUESTO
    _6_5_deudas_del_negocio
    _6_6_Dificultades_Comerciales
    _5_20_participa_organizacion
    _7_RELACIONES_VENTA_ALIMENTOS
    _7_1_Oferta_Alimentos_Produccio
    porcentaje_perdida_semana
    _7_2_produc_dejo_producir_vend
    VENTA_NUEVOS_ALIMENTOS
    _7_4_Actualmente_Usted_utiliza
    _7_6_canal_compra_comerciantes
    _9_2_brindo_apoyo
    _10_2_comerciante_problemas_pa
    problemas_paro
    resolucion_problema_paro


# Variables consumidor
    zona_carabayllo
    zona_pachacamac
    contacto_dirigente
    _3_4_tiempo_vive_hogar
    perdio_empleo
    ingreso_hogar
    participa_organiz_consumidores
    _7_RELACIONES_VENTA_ALIMENTOS
    _7_patrones_compra_consumidores
    _9_2_brindo_apoyo
    Patron_Consumo
    _10_2_consumidor_problemas_par
    problemas_paro
    resolucion_problema_paro
    direccion_hogar


In [57]:
df_main["_2019_perdio_empleo"]

0      NaN
1       si
2      NaN
3      NaN
4       no
      ... 
890    NaN
891    NaN
892     no
893    NaN
894    NaN
Name: _2019_perdio_empleo, Length: 895, dtype: object

In [60]:
df_main.loc[414].isna().sum()

814

In [62]:
df_main.loc[414][df_main.loc[414].notna()]

_index                                                       415
start                                 2022-08-30 23:42:59.163000
end                                   2022-08-30 23:43:25.236000
_1_5_perfil_entrevistado                              consumidor
__version__                               vwvoSDV79daQYxPddQWKi5
_version_                                 vAyg2NNtmiZULqujb5hCGE
_id                                                    180041604
_uuid                       94e0b9d7-3cbe-4fa9-8226-25f9e007a9b3
_submission_time                             2022-08-31 04:43:37
_status                                        submitted_via_web
_submitted_by                                             funsad
Name: 414, dtype: object

In [64]:
df_main = df_main.drop(414)

In [90]:
df_main[df_main["_1_4_1_ciudad"].isna()]

,_index,start,end,_1_1_codigo_cuestionario,_1_2_fecha_encuesta,_1_3_nombre_encuestador,_1_4_1_ciudad,_1_4_2_vecindario,sector_barrio_huerto,_1_5_perfil_entrevistado,...,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_altitude,_MAPEO_DE_LA_ZONA_DEL_A_para_llenar_luego_precision,_id,_uuid,_submission_time,_validation_status,_notes,_status,_submitted_by,_tags


In [70]:
df_main.loc[884, ["_1_4_1_ciudad", "_1_4_2_vecindario"]]

_1_4_1_ciudad               NaN
_1_4_2_vecindario    pachacamac
Name: 884, dtype: object

In [89]:
df_main.loc[884, "_1_4_1_ciudad"] = "lima"

array(['quito', 'lima'], dtype=object)

In [105]:
for ciudad in df_main["_1_4_1_ciudad"].unique():
    print(ciudad)
    dff = df_main[df_main["_1_4_1_ciudad"] == ciudad]
    unique_values = (dff["_3_3_2_Provincia"]
        .dropna()
        .str.lower()
        .str.strip()
        .str.normalize('NFKD')
        .str.encode('ascii', errors='ignore')
        .str.decode('utf-8')
        .str.replace('[^a-zA-Z ]', '')
        .unique()
    )
    unique_values.sort()
    print(unique_values)

quito
['azuay' 'bolivar' 'canar' 'caracas' 'carchi' 'chimborazo' 'chota'
 'comandancia de barinas' 'cotopaxi' 'cuenca' 'ecuador' 'el oro'
 'esmeraldas' 'guayaqui' 'guayaquil' 'guayas' 'ibarra' 'imbabura' 'loja'
 'los rios' 'manabi' 'manabia' 'manavi' 'morona santiago' 'napo' 'narino'
 'orellana' 'pastaza' 'pichicha' 'pichinca' 'pichincha' 'pillaro'
 'pueblo viejo' 'quito' 'riobamba' 'ruminahui' 'santa elena'
 'santo domingo' 'sto domingo' 'tulcan' 'tumgurahua' 'tunguragua'
 'tungurahua']
lima
['' 'abancay' 'amazonas' 'ancahs' 'ancash' 'andahuaylas' 'angaraes'
 'apurimac' 'arequipa' 'asuncion' 'asunsion' 'asunsionchacas' 'ayacucho'
 'barranca' 'bolivar' 'cajamarca' 'cajatambo' 'cajatambo  lima' 'callao'
 'canchis' 'canete' 'canta' 'carabaya' 'caracas' 'carhuaz' 'celendin'
 'cercado de lima' 'cerro de pasco' 'chachapoyas' 'chachapoyas amazonas'
 'chancay' 'chanchamayo junin' 'chepen' 'chiclayo' 'chota' 'chulcampa'
 'chulucanas' 'chupaca' 'coronel portillo' 'cusco' 'cuzco'
 'departamento 

In [ ]:
# Replace after cleaning _3_3_2_Provincia


{ # Ecuador
 'guayaqui': 'guayaquil',
 'pichicha': 'pichincha',
 'pichinca': 'pichincha',
 'sto domingo': 'santo domingo',
 'tumgurahua': 'tungurahua',
 'tunguragua':'tungurahua',
  # Peru
 'ancash': 'ancahs',
    
    'asuncion' 'asunsion' 'asunsionchacas'
    'cajatambo' 'cajatambo  lima' 'callao'
 'canchis' 'canete' 'canta' 'carabaya' 'caracas' 'carhuaz' 'celendin'
 'cercado de lima' 'cerro de pasco' 
    
    'ayacucho' 'departamento de ayacucho' 
    
    'chachapoyas' 'chachapoyas amazonas'
    'cusco' 'cuzco'
 'departamento ancash'  
    
 'huancayo' 'huancayo departamento junin' 
    
     'huari  ancash'
     'junin'
 'tarma departamento junin' 
    
     'la mar ayacucho' 
    'lamas  region san martin' 
    
     'lucana' 'lucanas' 
    
 'provincia ascope  region la libertad' 
}

In [28]:
for row in df_main["_3_3_2_Provincia"].str.lower().str.strip().value_counts().items():
    print(row)

('lima', 261)
('pichincha', 227)
('quito', 27)
('tungurahua', 18)
('cotopaxi', 18)
('chimborazo', 15)
('piura', 15)
('loja', 14)
('cajamarca', 10)
('ica', 10)
('canta', 9)
('arequipa', 7)
('ancash', 7)
('imbabura', 7)
('cañete', 7)
('bolivar', 7)
('huarochiri', 6)
('huancayo', 6)
('huanuco', 6)
('andahuaylas', 5)
('callao', 4)
('bolívar', 4)
('trujillo', 4)
('ayacucho', 4)
('guayas', 4)
('abancay', 4)
('puno', 4)
('san martin', 3)
('pichincha.', 3)
('junin', 3)
('manabí', 3)
('santo domingo', 3)
('huancavelica', 3)
('cuzco', 3)
('pachacamac', 3)
('caracas', 3)
('carchi', 3)
('el oro', 3)
('manabí.', 2)
('azuay', 2)
('carhuaz', 2)
('huánuco', 2)
('lambayeque', 2)
('cerro de pasco', 2)
('chota', 2)
('junín', 2)
('cañar', 2)
('santa rosa de quives', 2)
('pichinca', 2)
('huaraz', 2)
('huanta', 2)
('lamas - región san martín', 2)
('manabi', 2)
('amazonas', 2)
('los rios', 2)
('huari - ancash', 2)
('iquitos', 2)
('asunsion', 2)
('lucanas', 2)
('pataz', 1)
('puerto cabello', 1)
('apurimac', 1

In [ ]:
('lima', 261)
('pichincha', 227)
('quito', 27)
('tungurahua', 18)
('cotopaxi', 18)
('chimborazo', 15)
('piura', 15)
('loja', 14)
('cajamarca', 10)
('ica', 10)
('canta', 9)
('arequipa', 7)
('ancash', 7)
('imbabura', 7)
('cañete', 7)
('bolivar', 7)
('huarochiri', 6)
('huancayo', 6)
('huanuco', 6)
('andahuaylas', 5)
('callao', 4)
('bolívar', 4)
('trujillo', 4)
('ayacucho', 4)
('guayas', 4)
('abancay', 4)
('puno', 4)
('san martin', 3)
('pichincha.', 3)
('junin', 3)
('manabí', 3)
('santo domingo', 3)
('huancavelica', 3)
('cuzco', 3)
('pachacamac', 3)
('caracas', 3)
('carchi', 3)
('el oro', 3)
('manabí.', 2)
('azuay', 2)
('carhuaz', 2)
('huánuco', 2)
('lambayeque', 2)
('cerro de pasco', 2)
('chota', 2)
('junín', 2)
('cañar', 2)
('santa rosa de quives', 2)
('pichinca', 2)
('huaraz', 2)
('huanta', 2)
('lamas - región san martín', 2)
('manabi', 2)
('amazonas', 2)
('los rios', 2)
('huari - ancash', 2)
('iquitos', 2)
('asunsion', 2)
('lucanas', 2)
('pataz', 1)
('puerto cabello', 1)
('apurimac', 1)
('tulcán', 1)
('mariscal caceres', 1)
('rumiñahui', 1)
('pasco', 1)
('morropon', 1)
('asuncion', 1)
('san marcos', 1)
('portuguesa', 1)
('madre de dios', 1)
('riobamba', 1)
('barranca', 1)
('chulucanas', 1)
('chachapoyas', 1)
('nariño', 1)
('-', 1)
('loreto', 1)
('chepen', 1)
('lucana', 1)
('tunguragua', 1)
('celendín', 1)
('asunsion-chacas', 1)
('tarma', 1)
('cusco', 1)
('chupaca', 1)
('chachapoyas -amazonas', 1)
('la mar- ayacucho', 1)
('la libertad', 1)
('angaraes', 1)
('yaracuy', 1)
('pillaro', 1)
('chulcampa', 1)
('cercado de lima', 1)
('canchis', 1)
('recuay', 1)
('maynas', 1)
('melgar', 1)
('orellana', 1)
('chancay', 1)
('los ríos', 1)
('sandia', 1)
('guayaquil', 1)
('ibarra', 1)
('pueblo viejo', 1)
('tarma, departamento junín', 1)
('cajatambo - lima', 1)
('cuenca', 1)
('provincia ascope - región la libertad', 1)
('los ríos.', 1)
('napo', 1)
('coronel portillo', 1)
('sto domingo.', 1)
('paruro', 1)
('huaral', 1)
('pallasca', 1)
('chiclayo', 1)
('comandancia de barinas', 1)
('santa elena', 1)
('ancahs', 1)
('pichicha.', 1)
('departamento ancash', 1)
('esmeraldas.', 1)
('santa', 1)
('pastaza', 1)
('tacna', 1)
('essen', 1)
('yungay', 1)
('carabaya', 1)
('lampa', 1)
('panao- huánuco', 1)
('huaro', 1)
('huancayo, departamento junín', 1)
('chanchamayo (junin)', 1)
('manavi', 1)
('imbabura.', 1)
('manabía', 1)
('pichicha', 1)
('ecuador', 1)
('tumgurahua', 1)
('morona santiago', 1)
('departamento de ayacucho', 1)
('cajatambo', 1)
('guayaqui', 1)
('tocache', 1)
('santo domingo de olleros', 1)

In [23]:
df_main["_3_3_1_Distrito_Canton"].value_counts()

Quito                       212
Pachacamac                   48
Carabayllo                   39
Pichincha                    27
LIMA                         21
                           ... 
PACHAMAC                      1
Quitó                         1
Huancane                      1
San Simón                     1
Nuevo Progreso - Tocache      1
Name: _3_3_1_Distrito_Canton, Length: 299, dtype: int64

0                         Quito
1                         Quito
2                         Quito
3                         Quito
4                         Quito
                 ...           
890               Perdo Moncayo
891                     Píllaro
892    Nuevo Progreso - Tocache
893                 Jesús Maria
894                     Olleros
Name: _3_3_1_Distrito_Canton, Length: 895, dtype: object